In [6]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
# from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
# import numpy as np
# import sklearn
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

In [7]:
mean = np.load('feature/fbank4/mean.npy')
std = np.load('feature/fbank4/std.npy')
min_ = np.load('feature/fbank4/min.npy')
range_ = np.load('feature/fbank4/range.npy')
# autoencoder = load_model('model/lgd_dense_AE3.h5')
# X_un_dense = np.load('feature/fbank2/semi/X_un_dense.npy')
# Y_un_dense = np.load('feature/fbank2/semi/Y_un_dense.npy')
# X_test_dense = np.load('feature/fbank2/semi/X_test_dense.npy')
# Y_test_dense = np.load('feature/fbank2/semi/Y_test_dense.npy')

base_path = 'feature/fbank4/'#'/tmp2/b03902110/newphase1'
base_data_path = 'feature/fbank4/'#os.path.join(base_path, 'data')
num_fold = 10

In [8]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

def getTrainData():
    X = []
    y = []

    for i in range(num_fold):
        fileX = os.path.join(base_data_path, 'X/X' + str(i+1) + '.npy')
        fileY = os.path.join(base_data_path, 'y/y' + str(i+1) + '.npy')
        
        X.append(np.load(fileX))
        y.append(np.load(fileY))

    X = np.array(X)
    y = np.array(y)

    return X, y

def split_data(X, y, idx):
    X_train = []
    y_train = []
    
    for i in range(num_fold):
        if i == idx:
            X_val = X[i]
            y_val = y[i]
            continue
        if X_train == []:
            X_train = X[i]
            y_train = y[i]
        else:
            X_train = np.concatenate((X_train, X[i]))
            y_train = np.concatenate((y_train, y[i]))

    return X_train, y_train, X_val, y_val

def normalize(X_train, X_val):
    X_train = (X_train - mean)/(std)
#     X_train = (X_train - min_)/range_
    X_val = (X_val - mean)/(std)
#     X_val = (X_val - min_)/range_

    return X_train, X_val


In [9]:
def get_model():
    first_size = 11
    second_size = 8
    input_img = Input(shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3]))
    #block1
    conv = Conv2D(32,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(input_img)
    act = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, 
                shared_axes=None)(conv)
    # pad = ZeroPadding2D(padding=(2, 2), data_format='channels_last')(act)
    bn = BatchNormalization()(act)
    dp1 = Dropout(0.25)(bn)

    # add = Add()([])
    conv = Conv2D(32,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(dp1)
    act = LeakyReLU(alpha=0.05)(conv)
    bn = BatchNormalization()(act)
    # pad = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(bn)
    # maxpool = MaxPooling2D(pool_size=(2,2))(bn)
    dp2 = Dropout(0.3)(bn)

    # block2
    add = Add()([dp1,dp2])
    conv = Conv2D(32,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(add)
    act = LeakyReLU(alpha=0.33)(conv)
    bn = BatchNormalization()(act)
    # pad = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(bn)
    dp1 = Dropout(0.35)(bn)

    add = Add()([dp1,dp2])
    conv = Conv2D(32,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(add)
    act = LeakyReLU(alpha=0.33)(conv)
    # pad = ZeroPadding2D(padding=(2, 2), data_format='channels_last')(act)
    bn = BatchNormalization()(act)
    # maxpool = MaxPooling2D(pool_size=(2,2))(bn)
    dp2 = Dropout(0.4)(bn)

    #block 3
    add = Add()([dp1,dp2])
    conv = Conv2D(32,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(add)
    act = LeakyReLU(alpha=0.33)(conv)
    bn = BatchNormalization()(act)
    # pad = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(bn)
    dp1 = Dropout(0.45)(bn)

    add = Add()([dp1,dp2])
    conv = Conv2D(32,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(add)
    act = LeakyReLU(alpha=0.33)(conv)
    # pad = ZeroPadding2D(padding=(2, 2), data_format='channels_last')(act)
    bn = BatchNormalization()(act)
    # maxpool = MaxPooling2D(pool_size=(2,2))(bn)
    dp2 = Dropout(0.5)(bn)

    #block4
    add = Add()([dp1,dp2])
    conv = Conv2D(32,(int(first_size/2)+1,int(second_size/2)+1),padding='same',
                  kernel_initializer='glorot_normal')(add)
    bn = BatchNormalization()(conv)
    act = LeakyReLU(alpha=0.33)(bn)
    # pad = ZeroPadding2D(padding=(2, 2), data_format='channels_last')(act)
    dp1 = Dropout(0.55)(act)

    add = Add()([dp1,dp2])
    conv = Conv2D(32,(int(first_size/2)+1,int(second_size/2)+1),padding='same',
                  kernel_initializer='glorot_normal')(add)
    bn = BatchNormalization()(conv)
    act = LeakyReLU(alpha=0.33)(bn)
    # pad = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(bn)
    # avgpool = AveragePooling2D(pool_size=(1,4))(act)
    # maxpool = MaxPooling2D(pool_size=(2,2))(bn)
    cnn = Dropout(0.6)(act)
    shape = K.int_shape(cnn)

    # clf block
    x = Reshape((shape[1],shape[2]*shape[3]))(cnn)
    # x = TimeDistributed(Dense(128),input_shape=(11,2,256))(cnn)
    x = LSTM(1024,return_sequences=True,dropout=0.75,recurrent_dropout=0.75,
                             kernel_initializer='lecun_normal')(x)
    res1 = GRU(512,return_sequences=False,dropout=0.55,recurrent_dropout=0.55,
            kernel_initializer='lecun_normal')(x)
    # x = Concatenate()([x,encoder])
    x = BatchNormalization()(res1)
    '''
    for i in range(101):
        x_ = Dense(512, activation='selu', kernel_initializer='lecun_normal',
                     kernel_regularizer=l2(1e-4))(x)
        x_ = Dropout(0.3)(x_)
        x = Add()([x_, x])
        x = BatchNormalization()(x)

    '''
    for i in range(2):
        res2 = Dense(256, activation='selu', kernel_initializer='lecun_normal',
                     kernel_regularizer=l2(3e-4))(x)
        x = Dropout(0.45)(res2)
        x = Concatenate()([x,res1])
        x = BatchNormalization()(x)
        res1 = Dense(256, activation='selu', kernel_initializer='lecun_normal',
                     kernel_regularizer=l2(3e-4))(x)
        x = Dropout(0.45)(res1)
        x = Concatenate()([x,res2])
        x = BatchNormalization()(x)

    # res2 = Dense(256, activation='selu', kernel_initializer='lecun_normal')(x)
    # x = Dropout(0.25)(res2)
    # x = Concatenate()([x,res1])
    # x = BatchNormalization()(x)
    # res1 = Dense(256, activation='selu', kernel_initializer='lecun_normal')(x)
    # x = Dropout(0.25)(res1)
    # x = Concatenate()([x,res2])
    # x = BatchNormalization()(x)
    # res2 = Dense(128, activation='selu', kernel_initializer='lecun_normal')(x)
    # x = Dropout(0.25)(res2)
    # x = Concatenate()([x,res1])
    # x = BatchNormalization()(x)
    x = Dense(64, activation='selu', kernel_initializer='lecun_normal')(x)
    # x = Dropout(0.25)(x)
    clf = Dense(41,activation='softmax',name='clf')(x)

    model = Model(inputs=input_img, outputs=clf)
    # model = Model(input_img,decoder)
    model.summary()
    return model

In [10]:
# val_set_num = [x+1 for x in range(num_fold)]#str(sys.argv[1])
val_set_num = [0]#str(sys.argv[1])
min_ = np.swapaxes(min_,0,1)
mean = np.swapaxes(mean,0,1)
range_ = np.swapaxes(range_,0,1)
std = np.swapaxes(std,0,1)
for fold in val_set_num:
    X, y = getTrainData()
    X = np.swapaxes(X,2,3)
    X_train, Y_train, X_valid, Y_valid = split_data(X, y, fold) #fold
    X_train, X_valid = normalize(X_train, X_valid)
    print(X_train.shape, Y_train.shape)

    model = get_model()
    batchSize=[128,256]
    batchSize = random.choice(batchSize)
    patien=100
    epoch=3000
    saveP = 'model/LGD_ConvLSTM_clf_'+str(fold)+'.h5'

    model.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=3e-3,decay=1e-8),
                  metrics=['acc']) 

    callback=[
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=int(patien/2),
                                      min_lr=1e-4,mode='min'),
        EarlyStopping(patience=patien,monitor='val_acc',verbose=1,mode='max'),
        ModelCheckpoint(saveP,monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,mode='max'),
    ]

    model.fit(X_train,Y_train,
              shuffle=True,
              callbacks=callback, 
              class_weight='auto',
              validation_data=(X_valid,Y_valid),
              batch_size=batchSize,
              epochs=epoch)
#1 =>0.41
#2 =>0.51482
#3 =>0.45822
#4 =>0.49596
#5 =>0.50674
#6 =>0.33154
#7 =>0.54717
#8 => 0.37736
#9 => 0.40431
#0 => 0.41240

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(3339, 88, 64, 1) (3339, 41)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 88, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 88, 64, 32)   2848        input_1[0][0]                    
__________________________________________________________________________________________________
p_re_lu_1 (PReLU)               (None, 88, 64, 32)   180224      conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 88, 64, 32)   128         p_re_lu_1[0][0]                  
________________________________________________________________________________

Train on 3339 samples, validate on 371 samples
Epoch 1/3000
3339/3339 [==============================] - 22s 7ms/step - loss: 4.3700 - acc: 0.0662 - val_loss: 5.0125 - val_acc: 0.0809

Epoch 00001: val_acc improved from -inf to 0.08086, saving model to model/LGD_ConvLSTM_clf_0.h5
Epoch 2/3000
3339/3339 [==============================] - 17s 5ms/step - loss: 3.7753 - acc: 0.1168 - val_loss: 5.2678 - val_acc: 0.0836

Epoch 00002: val_acc improved from 0.08086 to 0.08356, saving model to model/LGD_ConvLSTM_clf_0.h5
Epoch 3/3000
3339/3339 [==============================] - 17s 5ms/step - loss: 3.5448 - acc: 0.1444 - val_loss: 3.9493 - val_acc: 0.1402

Epoch 00003: val_acc improved from 0.08356 to 0.14016, saving model to model/LGD_ConvLSTM_clf_0.h5
Epoch 4/3000
3339/3339 [==============================] - 17s 5ms/step - loss: 3.3922 - acc: 0.1692 - val_loss: 3.6588 - val_acc: 0.1995

Epoch 00004: val_acc improved from 0.14016 to 0.19946, saving model to model/LGD_ConvLSTM_clf_0.h5
Epoch 5/

3339/3339 [==============================] - 18s 5ms/step - loss: 2.2290 - acc: 0.4037 - val_loss: 2.5631 - val_acc: 0.3369

Epoch 00042: val_acc improved from 0.33154 to 0.33693, saving model to model/LGD_ConvLSTM_clf_0.h5
Epoch 43/3000
3339/3339 [==============================] - 17s 5ms/step - loss: 2.1117 - acc: 0.4349 - val_loss: 3.2246 - val_acc: 0.2722

Epoch 00043: val_acc did not improve from 0.33693
Epoch 44/3000
3339/3339 [==============================] - 17s 5ms/step - loss: 2.1411 - acc: 0.4355 - val_loss: 3.9910 - val_acc: 0.1887

Epoch 00044: val_acc did not improve from 0.33693
Epoch 45/3000
3339/3339 [==============================] - 17s 5ms/step - loss: 2.2059 - acc: 0.4133 - val_loss: 2.5860 - val_acc: 0.3423

Epoch 00045: val_acc improved from 0.33693 to 0.34232, saving model to model/LGD_ConvLSTM_clf_0.h5
Epoch 46/3000
3339/3339 [==============================] - 18s 5ms/step - loss: 2.1793 - acc: 0.4166 - val_loss: 2.7706 - val_acc: 0.2830

Epoch 00046: val_acc 

3339/3339 [==============================] - 18s 5ms/step - loss: 2.2963 - acc: 0.3851 - val_loss: 6.8779 - val_acc: 0.0863

Epoch 00084: val_acc did not improve from 0.41240
Epoch 85/3000
3339/3339 [==============================] - 17s 5ms/step - loss: 2.6305 - acc: 0.3040 - val_loss: 3.6120 - val_acc: 0.1402

Epoch 00085: val_acc did not improve from 0.41240
Epoch 86/3000
3339/3339 [==============================] - 17s 5ms/step - loss: 2.6739 - acc: 0.3010 - val_loss: 3.3840 - val_acc: 0.1779

Epoch 00086: val_acc did not improve from 0.41240
Epoch 87/3000
3339/3339 [==============================] - 17s 5ms/step - loss: 2.6536 - acc: 0.2929 - val_loss: 3.0475 - val_acc: 0.2237

Epoch 00087: val_acc did not improve from 0.41240
Epoch 88/3000
3339/3339 [==============================] - 18s 5ms/step - loss: 2.6253 - acc: 0.3004 - val_loss: 3.1352 - val_acc: 0.2291

Epoch 00088: val_acc did not improve from 0.41240
Epoch 89/3000
3339/3339 [==============================] - 18s 5ms/st

In [5]:
first_size = 8
second_size = 8
input_img = Input(shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3]))
#block1
conv = Conv2D(32,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(input_img)
act = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, 
            shared_axes=None)(conv)
bn = BatchNormalization()(act)
dp = Dropout(0.25)(bn)

conv = Conv2D(32,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(dp)
act = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, 
            shared_axes=None)(conv)
bn = BatchNormalization()(act)
maxpool = MaxPooling2D(pool_size=(2,2))(bn)
dp = Dropout(0.25)(maxpool)

# block2
conv = Conv2D(64,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(dp)
act = LeakyReLU(alpha=0.33)(conv)
bn = BatchNormalization()(act)
dp = Dropout(0.25)(bn)

conv = Conv2D(64,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(dp)
act = LeakyReLU(alpha=0.33)(conv)
bn = BatchNormalization()(act)
maxpool = MaxPooling2D(pool_size=(2,2))(bn)
dp = Dropout(0.25)(maxpool)

#block 3
conv = Conv2D(128,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(dp)
act = LeakyReLU(alpha=0.33)(conv)
bn = BatchNormalization()(act)
dp = Dropout(0.25)(bn)

conv = Conv2D(128,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(dp)
act = LeakyReLU(alpha=0.33)(conv)
bn = BatchNormalization()(act)
maxpool = MaxPooling2D(pool_size=(2,2))(bn)
dp = Dropout(0.25)(maxpool)

#block4
conv = Conv2D(256,(int(first_size/2),int(second_size/2)),padding='same',
              kernel_initializer='glorot_normal')(dp)
bn = BatchNormalization()(conv)
act = LeakyReLU(alpha=0.33)(bn)
dp = Dropout(0.25)(act)

conv = Conv2D(256,(int(first_size/2),int(second_size/2)),padding='same',
              kernel_initializer='glorot_normal')(dp)
bn = BatchNormalization()(conv)
act = LeakyReLU(alpha=0.33)(bn)
avgpool = AveragePooling2D(pool_size=(1,4))(act)
# maxpool = MaxPooling2D(pool_size=(2,2))(bn)
cnn = Dropout(0.25)(avgpool)


# AUTO ENCODER BLOCK
shape = K.int_shape(cnn)
flat = Flatten()(cnn)
x = Dense(4096)(flat)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.33)(x)
encoder = Dense(2048,name='encoder',activation='selu')(x)
d = Dense(4096)(encoder)
d = LeakyReLU(alpha=0.33)(d)
d = Dense(shape[1] * shape[2] * shape[3])(d)
d = Reshape((shape[1], shape[2], shape[3]))(d)


d = UpSampling2D((1,4))(d)
d = Conv2DTranspose(256,(int(first_size/2),int(second_size/2)),padding='same',
                    kernel_initializer='glorot_normal')(d)
d = LeakyReLU(alpha=0.33)(d)
d = Conv2DTranspose(256,(int(first_size/2),int(second_size/2)),padding='same',
                    kernel_initializer='glorot_normal')(d)
d = LeakyReLU(alpha=0.33)(d)
d = UpSampling2D((2,2))(d)
d = Conv2DTranspose(128,(first_size,second_size),padding='same',
                    kernel_initializer='glorot_normal')(d)
d = LeakyReLU(alpha=0.33)(d)
d = Conv2DTranspose(128,(first_size,second_size),padding='same',
                    kernel_initializer='glorot_normal')(d)
d = LeakyReLU(alpha=0.33)(d)
d = UpSampling2D((2,2))(d)
d = Conv2DTranspose(64,(first_size,second_size),padding='same',
                    kernel_initializer='glorot_normal')(d)
d = LeakyReLU(alpha=0.33)(d)
d = Conv2DTranspose(64,(first_size,second_size),padding='same',
                    kernel_initializer='glorot_normal')(d)
d = LeakyReLU(alpha=0.33)(d)
d = UpSampling2D((2,2))(d)
d = Conv2DTranspose(32,(first_size,second_size),padding='same',
                    kernel_initializer='glorot_normal')(d)
d = LeakyReLU(alpha=0.33)(d)
d = Conv2DTranspose(32,(first_size,second_size),padding='same',
                    kernel_initializer='glorot_normal')(d)
d = LeakyReLU(alpha=0.33)(d)
decoder = Conv2DTranspose(1,(first_size,second_size),padding='same',
                          kernel_initializer='glorot_normal',name='AE')(d)
#  = LeakyReLU(name='AE',alpha=0.33)(d)
# d = Conv2DTranspose(1,(first_size,second_size),padding='same',kernel_initializer='glorot_normal')(d)
# decoder = LeakyReLU(alpha=0.33)(d)


# clf block
x = Reshape((shape[1],shape[2]*shape[3]))(cnn)
# x = TimeDistributed(Dense(128),input_shape=(11,2,256))(cnn)
x = LSTM(512,return_sequences=True,dropout=0.55,recurrent_dropout=0.55,
                         kernel_initializer='lecun_normal')(x)
res1 = GRU(512,return_sequences=False,dropout=0.55,recurrent_dropout=0.55,
        kernel_initializer='lecun_normal')(x)
# x = Concatenate()([x,encoder])
x = BatchNormalization()(res1)
'''
for i in range(101):
    x_ = Dense(512, activation='selu', kernel_initializer='lecun_normal',
                 kernel_regularizer=l2(1e-4))(x)
    x_ = Dropout(0.3)(x_)
    x = Add()([x_, x])
    x = BatchNormalization()(x)

'''
for i in range(16):
    res2 = Dense(512, activation='selu', kernel_initializer='lecun_normal',
                 kernel_regularizer=l2(5e-5))(x)
    x = Dropout(0.28)(res2)
    x = Concatenate()([x,res1])
    x = BatchNormalization()(x)
    res1 = Dense(512, activation='selu', kernel_initializer='lecun_normal',
                 kernel_regularizer=l2(5e-5))(x)
    x = Dropout(0.28)(res1)
    x = Concatenate()([x,res2])
    x = BatchNormalization()(x)

# res2 = Dense(256, activation='selu', kernel_initializer='lecun_normal')(x)
# x = Dropout(0.25)(res2)
# x = Concatenate()([x,res1])
# x = BatchNormalization()(x)
# res1 = Dense(256, activation='selu', kernel_initializer='lecun_normal')(x)
# x = Dropout(0.25)(res1)
# x = Concatenate()([x,res2])
# x = BatchNormalization()(x)
# res2 = Dense(128, activation='selu', kernel_initializer='lecun_normal')(x)
# x = Dropout(0.25)(res2)
# x = Concatenate()([x,res1])
# x = BatchNormalization()(x)
# x = Dense(128, activation='selu', kernel_initializer='lecun_normal')(x)
# x = Dropout(0.25)(x)
clf = Dense(41,activation='softmax',name='clf')(x)

model = Model(inputs=input_img, outputs=clf)
# model = Model(input_img,decoder)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 88, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 88, 64, 32)   2080        input_1[0][0]                    
__________________________________________________________________________________________________
p_re_lu_1 (PReLU)               (None, 88, 64, 32)   180224      conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 88, 64, 32)   128         p_re_lu_1[0][0]                  
__________________________________________________________________________________________________
dropout_1 

In [6]:
model.compile(loss=['categorical_crossentropy'],optimizer=Adam(),metrics=['acc']) #,loss_weights=[0,1
model.fit(X_train,Y_train,
          shuffle=True,
          validation_data=(X_valid,Y_valid),
          batch_size=128,epochs=500)

Train on 3339 samples, validate on 371 samples
Epoch 1/500
3339/3339 [==============================] - 25s 8ms/step - loss: 4.8642 - acc: 0.0407 - val_loss: 6.3721 - val_acc: 0.0674
Epoch 2/500
3339/3339 [==============================] - 8s 3ms/step - loss: 4.2255 - acc: 0.0656 - val_loss: 5.6264 - val_acc: 0.0458
Epoch 3/500
3339/3339 [==============================] - 8s 3ms/step - loss: 3.9139 - acc: 0.0848 - val_loss: 5.0452 - val_acc: 0.0997
Epoch 4/500
3339/3339 [==============================] - 9s 3ms/step - loss: 3.7673 - acc: 0.0964 - val_loss: 4.5385 - val_acc: 0.0836
Epoch 5/500
3339/3339 [==============================] - 8s 3ms/step - loss: 3.7071 - acc: 0.1045 - val_loss: 4.8070 - val_acc: 0.0970
Epoch 6/500
3339/3339 [==============================] - 9s 3ms/step - loss: 3.6081 - acc: 0.1066 - val_loss: 3.9529 - val_acc: 0.1240
Epoch 7/500
3339/3339 [==============================] - 8s 3ms/step - loss: 3.6355 - acc: 0.0982 - val_loss: 4.2757 - val_acc: 0.1024
Epoch 8

Epoch 61/500
3339/3339 [==============================] - 8s 2ms/step - loss: 2.9149 - acc: 0.2366 - val_loss: 6.0667 - val_acc: 0.1644
Epoch 62/500
3339/3339 [==============================] - 8s 3ms/step - loss: 2.7993 - acc: 0.2579 - val_loss: 5.2201 - val_acc: 0.0970
Epoch 63/500
3339/3339 [==============================] - 8s 3ms/step - loss: 2.8927 - acc: 0.2405 - val_loss: 8.9181 - val_acc: 0.1132
Epoch 64/500
3339/3339 [==============================] - 9s 3ms/step - loss: 2.8886 - acc: 0.2354 - val_loss: 7.8924 - val_acc: 0.1348
Epoch 65/500
3339/3339 [==============================] - 8s 3ms/step - loss: 2.7800 - acc: 0.2522 - val_loss: 8.2023 - val_acc: 0.0809
Epoch 66/500
3339/3339 [==============================] - 8s 3ms/step - loss: 2.7240 - acc: 0.2656 - val_loss: 6.0890 - val_acc: 0.1078
Epoch 67/500
3339/3339 [==============================] - 8s 2ms/step - loss: 2.6917 - acc: 0.2776 - val_loss: 6.4981 - val_acc: 0.1456
Epoch 68/500
3339/3339 [========================

3339/3339 [==============================] - 9s 3ms/step - loss: 2.5216 - acc: 0.3211 - val_loss: 5.4673 - val_acc: 0.1402
Epoch 122/500
3339/3339 [==============================] - 9s 3ms/step - loss: 2.5307 - acc: 0.3166 - val_loss: 9.8161 - val_acc: 0.1132
Epoch 123/500
3339/3339 [==============================] - 8s 3ms/step - loss: 2.4718 - acc: 0.3474 - val_loss: 9.3834 - val_acc: 0.1429
Epoch 124/500
3339/3339 [==============================] - 9s 3ms/step - loss: 2.4822 - acc: 0.3306 - val_loss: 8.3860 - val_acc: 0.1590
Epoch 125/500
3339/3339 [==============================] - 8s 3ms/step - loss: 2.4884 - acc: 0.3351 - val_loss: 4.5710 - val_acc: 0.1887
Epoch 126/500
3339/3339 [==============================] - 9s 3ms/step - loss: 2.4207 - acc: 0.3543 - val_loss: 2.9720 - val_acc: 0.2695
Epoch 127/500
3339/3339 [==============================] - 9s 3ms/step - loss: 2.4135 - acc: 0.3588 - val_loss: 7.5263 - val_acc: 0.1240
Epoch 128/500
3339/3339 [==============================

KeyboardInterrupt: 

In [22]:
first_size = 8
second_size = 8
input_img = Input(shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3]))
#block1
conv = Conv2D(32,(first_size,second_size),padding='same',activation='relu')(input_img)
conv = Conv2D(32,(first_size,second_size),padding='same',activation='relu')(conv)
maxpool = MaxPooling2D(pool_size=(2,2))(conv)


# block2
conv = Conv2D(64,(first_size,second_size),padding='same',activation='relu')(maxpool)
conv = Conv2D(64,(first_size,second_size),padding='same',activation='relu')(conv)
cnn = MaxPooling2D(pool_size=(2,2))(conv)


#block 3
# conv = Conv2D(128,(first_size,second_size),padding='same',activation='relu')(maxpool)
# conv = Conv2D(128,(first_size,second_size),padding='same',activation='relu')(conv)
# cnn = MaxPooling2D(pool_size=(2,2))(conv)

#block4
# conv = Conv2D(256,(int(first_size/2),int(second_size/2)),padding='same',activation='relu' )(maxpool)
# conv = Conv2D(256,(int(first_size/2),int(second_size/2)),padding='same',activation='relu')(conv)
# cnn = AveragePooling2D(pool_size=(1,4))(conv)



# AUTO ENCODER BLOCK
shape = K.int_shape(cnn)
flat = Flatten()(cnn)
x = Dense(4096,activation='relu')(flat)
encoder = Dense(2048,name='encoder')(x)
d = Dense(4096,activation='relu')(encoder)
d = Dense(shape[1] * shape[2] * shape[3])(d)
d = Reshape((shape[1], shape[2], shape[3]))(d)

# d = UpSampling2D((1,4))(d)
# d = Conv2DTranspose(256,(int(first_size/2),int(second_size/2)),padding='same',activation='relu')(d)
# d = Conv2DTranspose(256,(int(first_size/2),int(second_size/2)),padding='same',activation='relu')(d)

d = UpSampling2D((2,2))(d)
d = Conv2DTranspose(128,(first_size,second_size),padding='same',activation='relu')(d)
d = Conv2DTranspose(128,(first_size,second_size),padding='same',activation='relu')(d)
d = UpSampling2D((2,2))(d)
d = Conv2DTranspose(64,(first_size,second_size),padding='same',activation='relu')(d)
d = Conv2DTranspose(64,(first_size,second_size),padding='same',activation='relu')(d)
d = UpSampling2D((2,2))(d)
d = Conv2DTranspose(32,(first_size,second_size),padding='same',activation='relu')(d)
d = Conv2DTranspose(32,(first_size,second_size),padding='same',activation='relu')(d)
decoder = Conv2DTranspose(1,(first_size,second_size),padding='same',name='AE')(d)
#  = LeakyReLU(name='AE',alpha=0.33)(d)
# d = Conv2DTranspose(1,(first_size,second_size),padding='same')(d)
# decoder = LeakyReLU(alpha=0.33)(d)


# clf block
# x = Reshape((shape[1]*shape[2]*shape[3]))(cnn)
x = Flatten()(cnn)
# x = TimeDistributed(Dense(128),input_shape=(11,2,256))(cnn)
# x = LSTM(512,return_sequences=True)(x)
# x = GRU(512,return_sequences=False)(x)
# x = Concatenate()([x,encoder])
x = Dense(2048, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
clf = Dense(41,activation='softmax',name='clf')(x)

model = Model(inputs=input_img, outputs=clf)
# model = Model(input_img,decoder)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 88, 64, 1)         0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 88, 64, 32)        2080      
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 88, 64, 32)        65568     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 44, 32, 32)        0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 44, 32, 64)        131136    
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 44, 32, 64)        262208    
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 22, 16, 64)        0         
__________

In [ ]:
model.predict(X_valid)

In [13]:
Y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)